In [1]:
# Setup iniziale con i vari import da sfruttare per i due svolgimenti separati
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import classification_report

In [34]:
# Loading Dataset Kaggle
data = pd.read_csv('AirQualityUCI.csv', delimiter=';')
# Convert column 'Date' in datetime format to extract day, month, year, column 'Time' to pick hour
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')
# Replace dots with colons in the Time column
data['Time'] = pd.to_timedelta(data['Time'].str.replace('.', ':'))
# Create column 'datetime'
data['Datetime'] = data['Date'] + data['Time']
data.set_index('Datetime', inplace=True)
data.drop(columns=['Date', 'Time'], inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9471 entries, 2004-03-10 18:00:00 to NaT
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CO(GT)         9357 non-null   object 
 1   PT08.S1(CO)    9357 non-null   float64
 2   NMHC(GT)       9357 non-null   float64
 3   C6H6(GT)       9357 non-null   object 
 4   PT08.S2(NMHC)  9357 non-null   float64
 5   NOx(GT)        9357 non-null   float64
 6   PT08.S3(NOx)   9357 non-null   float64
 7   NO2(GT)        9357 non-null   float64
 8   PT08.S4(NO2)   9357 non-null   float64
 9   PT08.S5(O3)    9357 non-null   float64
 10  T              9357 non-null   object 
 11  RH             9357 non-null   object 
 12  AH             9357 non-null   object 
 13  Unnamed: 15    0 non-null      float64
 14  Unnamed: 16    0 non-null      float64
dtypes: float64(10), object(5)
memory usage: 1.2+ MB


In [35]:
data.head(5)

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
Datetime,,,,,,,,,,,,,,,
2004-03-10 18:00:00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578",NaN,NaN
2004-03-10 19:00:00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255",NaN,NaN
2004-03-10 20:00:00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502",NaN,NaN
2004-03-10 21:00:00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867",NaN,NaN
2004-03-10 22:00:00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888",NaN,NaN


In [36]:
# Preprocessing and 'Feature Selection'
data = data.dropna(axis=1, how='all')
data = data.select_dtypes(include=['datetime64[ns]', 'float64'])

In [37]:
data['NO2(GT)'].describe()

count    9357.000000
mean       58.148873
std       126.940455
min      -200.000000
25%        53.000000
50%        96.000000
75%       133.000000
max       340.000000
Name: NO2(GT), dtype: float64

In [38]:
global_mean = data['NO2(GT)'].mean()

In [39]:
daily_mean = data['NO2(GT)'].resample('D').mean()
weekly_mean = data['NO2(GT)'].resample('W').mean()

In [40]:
data['Media_Settimanale'] = weekly_mean.reindex(data.index).ffill()
data['Media_Giornaliera'] = daily_mean.reindex(data.index).ffill()
data['Media_Globale'] = global_mean

In [ ]:
data["target_settimanale"] = (data["NO2(GT)"] > data["Media_Settimanale"]).astype(int)
data["target_giornaliero"] = (data["NO2(GT)"] > data["Media_Giornaliera"]).astype(int)
data["target_globale"] = (data["NO2(GT)"] > data["Media_Globale"]).astype(int)